In [1]:
!pip install --upgrade --force-reinstall --no-cache-dir torch==2.1.0 triton --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[cu121-ampere] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 236.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 238.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 299.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 357.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 315.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 340.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 175.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 358.1 MB/s eta 0:00:00
  Attempting uninstall: mpmath
    Found existing installation: mpmath 1.3.0
    Uninstalling mpmath-1.3.0:
      Successfully uninstalled mpmath-1.3.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensi

In [2]:
# Import necessary libraries
hf_token = "XXX"
from unsloth import FastLanguageModel
import os
from google.colab import drive
import csv
from transformers import TextStreamer

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:112: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
# Load articles from Google Drive
def load_articles(directory_path):
    articles = {}
    titles = {}
    for file in os.listdir(directory_path):
        if file.endswith(".txt"):
            with open(os.path.join(directory_path, file), 'r', encoding='utf-8') as f:
                content = f.read().splitlines()
                title = content[0].strip()
                text = " ".join(content[2:]).strip()
                articles[file] = text
                titles[file] = title
    return articles, titles

In [4]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load articles
directory_path = "/content/drive/MyDrive/ColabNotebooks/CodePenal/"
articles, titles = load_articles(directory_path)
print(f"{len(articles)} articles loaded.")

434 articles loaded.


In [6]:
# Load the model
model_name = "unsloth/Phi-3.5-mini-instruct-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    token=hf_token,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.22.post7. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [7]:
# Enable faster inference
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32064, 3072)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LongRopeRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-0

In [8]:
# Alpaca prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [13]:
# Function to summarize an article and generate keywords
def summarize_article(article_text, article_title):
    instruction_summary = "Résume l'article suivant du code pénal avec le moins de mots possible:"
    instruction_keywords = "Génère 5 mots clés (sans les explications) permettant de classifier cet article de loi:"

    # Generate summary
    prompt_summary = alpaca_prompt.format(instruction_summary, article_text, "")
    inputs_summary = tokenizer([prompt_summary], return_tensors="pt").to("cuda")
    output_summary = model.generate(**inputs_summary, max_new_tokens=500)

    # Generate keywords
    prompt_keywords = alpaca_prompt.format(instruction_keywords, article_text, "")
    inputs_keywords = tokenizer([prompt_keywords], return_tensors="pt").to("cuda")
    output_keywords = model.generate(**inputs_keywords, max_new_tokens=50)

    # Decode the generated tensors into text
    summary_text = tokenizer.decode(output_summary[0], skip_special_tokens=True)
    keywords_text = tokenizer.decode(output_keywords[0], skip_special_tokens=True)

    # Return only the generated responses
    return summary_text, keywords_text

In [10]:
# Function to save results to a CSV file
def save_data_to_csv(csv_file, data):
    # Write data to the CSV file
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(["Index", "Title", "Summary", "Keywords"])
        # Write each row of data
        for index, (title, summary, keywords) in enumerate(data):
            writer.writerow([index + 1, title, summary, keywords])

In [14]:
#List to store data before writing to CSV
data_to_save = []

# Iterate through the first 5 articles and generate summaries and keywords
for i, (file, text) in enumerate(articles.items()):
    if i >= 5:
        break
    print(f"\nProcessing {titles[file]}:")

    # Generate summary and keywords
    summary, keywords = summarize_article(text, titles[file])

    # Add the results to the list for saving
    data_to_save.append((titles[file], summary, keywords))

# Specify the output CSV file
output_csv_file = "/content/drive/MyDrive/ColabNotebooks/CodePenal/summaries_keywords.csv"

# Save the results to the CSV file
save_data_to_csv(output_csv_file, data_to_save)

print(f"The results have been saved to {output_csv_file}")


Processing Pas de sanction sans loi:

Processing Conditions de temps:

Processing Crimes ou délits commis en Suisse:

Processing Crimes ou délits commis à l’étranger contre l’État:

Processing Infractions commises à l’étranger sur des mineurs:
The results have been saved to /content/drive/MyDrive/ColabNotebooks/CodePenal/summaries_keywords.csv
